IMDb 電影資料庫
電影評論數據集   
50000篇文字評論並且被標記為正或是負
我們試著用將影評訓練集和標籤來建立模型並利用影評測試集計算正評和負評的正確率

In [2]:
# 到這邊算是整理好資料的格式了  之後從這邊讀資料就好了
import pandas as pd


df = pd.read_csv('./movie_data.csv', encoding='utf-8')

df.head(10)



,review,sentiment
0,election is a chinese mob movie or triads in t...,1
1,i was just watching a forensic files marathon ...,0
2,police story is a stunning series of set piece...,1
3,dear readers the final battle between the rebe...,1
4,i have seen the perfect son about three times ...,1
5,a brilliant portrait of a traitor victor mclag...,1
6,if ever a potential movie must ve sounded like...,1
7,i d always wanted david duchovney to go into t...,1
8,perhaps if only to laugh at the way my favorit...,0
9,even though the story is light the movie flows...,1


# 處理較大的數據 - out-of-core learning

上次那 50000 筆評論
製作特徵向量以及分類還有調整參數 
是相當耗時的

為處理數據量過大而導致無法一次地儲存在Main Memory，
Scikit-Learn能讓系統做一些調整，out-of-core learning便是其中一個策略。


Out-of-core (or “external memory”) learning 
適用在data過大,無法存放於主記憶體(RAM) 時

以串流(stream)的方式從硬碟中讀取資料,批次處理

但不是每種演算法都可以這麼做
可以實作的項目如下: 
1.特徵提取 
    sklearn.feature_extraction.text.HashingVectorizer for text documents.
2.estimators:
    Classification
        sklearn.naive_bayes.MultinomialNB
        sklearn.naive_bayes.BernoulliNB
        sklearn.linear_model.Perceptron
        sklearn.linear_model.SGDClassifier
        sklearn.linear_model.PassiveAggressiveClassifier
    Regression
        sklearn.linear_model.SGDRegressor
        sklearn.linear_model.PassiveAggressiveRegressor
    Clustering
        sklearn.cluster.MiniBatchKMeans
    Decomposition / feature Extraction
        sklearn.decomposition.MiniBatchDictionaryLearning
        sklearn.decomposition.IncrementalPCA
        sklearn.decomposition.LatentDirichletAllocation
        sklearn.cluster.MiniBatchKMeans
        
而此時要使用 partial_fit 取代原先 fit 的功能

In [3]:
import numpy as np
import re
from nltk.corpus import stopwords

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

# 每次讀取並回傳一個doc
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
#            print (text, label) 
            yield text, label
            
#yield 和 return 很像，只是當函數呼叫 return 時，
#該函數 call stack (python 中是 frame) 就會被清除，
#程式主導權回到呼叫該函數的手上。 
#而 yield 會把程式主導權交給呼叫該函數的手上，但是他不會把函數的 
#call stack 清除，因此下次呼叫時，可以從上次未執行的部分開始執行，
#而不是重新建立一個新 stack。

#內建函數 (function) next() ，回傳參數 (parameter) 迭代器中下一個數值


# 備註:next() & yield-------------------------------------------------------

In [21]:
# list 
L = [x * x for x in range(10)]
L


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [24]:
# 把一個列表生成式的[]改成()，就創建了一個generator,從而節省大量的空間： 
g = (x * x for x in range(10))
g

<generator object <genexpr> at 0x000001BCBC52B3B8>

In [25]:
# 要一個一個print出來，可以通過next()函數獲得generator的下一個返回值
next(g)

0

In [28]:
next(g)
# generator保存的是算法，每次調用next(g)，就計算出g的下一個元素的值，直到計算到最後一個元素，沒有更多的元素時，拋出StopIteration的錯誤。

# 但正常是這樣用 
#for n in g:
#     print(n)

9

In [31]:
# fib函數實際上是定義了斐波拉契數列的推算規則
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        print(b)
        a, b = b, a + b
        n = n + 1
    return 'done'

In [30]:
fib(6)

1
1
2
3
5
8


'done'

In [34]:
# 把fib函數變成generator，只需要把print(b)改為yield b就可以了：
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1
    return 'done'

#函數是順序執行，遇到return語句或者最後一行函數語句就返回。
#generator的函數，在每次調用next()的時候執行，遇到yield語句返回，
#再次執行時從上次返回的yield語句處繼續執行。

In [33]:
f = fib(6)
f

<generator object fib at 0x000001BCBC52BD00>

In [39]:
next(f)

5

In [41]:
next(f)

StopIteration: done

# -------------------------------------------------------

In [4]:
stream_docs(path='./movie_data.csv')

<generator object stream_docs at 0x000002B996ABD678>

In [5]:
next(stream_docs(path='./movie_data.csv'))

# 這樣就可以讀出第一份的文件  還有標籤

('election is a chinese mob movie or triads in this case every two years an election is held to decide on a new leader and at first it seems a toss up between big d tony leung ka fai or as i know him the other tony leung and lok simon yam who was judge in full contact though once lok wins big d refuses to accept the choice and goes to whatever lengths he can to secure recognition as the new leader unlike any other asian film i watch featuring gangsters this one is not an action movie it has its bloody moments when necessary as in goodfellas but it s basically just a really effective drama there are a lot of characters which is really hard to keep track of but i think that plays into the craziness of it all a bit a 100 year old baton which is the symbol of power i mentioned before changes hands several times before things settle down and though it may appear that the film ends at the 65 or 70 minute mark there are still a couple big surprises waiting simon yam was my favorite character 

In [6]:
# 一次會回傳　doc_stream　中的sizez份數
# 輸入是串流 和份數

def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [7]:
#CountVectorizer,TfidfVectorizer 需要將完整詞彙放在主記憶體中，無法做out-of-core learning
#因此用　HashingVectorizer　會犧牲一點點準確度　

#Linear classifiers (SVM, logistic regression, a.o.) with SGD training.
#This estimator implements regularized linear models with stochastic gradient descent (SGD) learning

# n_features=2**21,hash之後的特徵的數目
# 選大一點是為了避免 hash碰撞, 但這樣會增加邏輯斯迴歸的係數

# decode_error 如果有含不是給定編碼的字符的字節序列。 
# 預設情況下，意味著將會引發UnicodeDecodeError。 讓我們忽略他:ignore 。 
# tokenizer : 用前面定義的函數
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='./movie_data.csv')

備註 :  短網址在數學上 (或者密碼學上) 是一種雜湊函數的應用： 讓一個任意長度的字串轉換成固定長度、特定符號組成的字串。所以理論上， 一定會有機會發生碰撞，原因在於雜湊函數是把較大集合的內容， 對映到較小的集合當中。
HashingVectorizer

In [8]:
import pyprind
pbar = pyprind.ProgBar(45)

from nltk.corpus import stopwords
# 停用字刪除 and, a...
stop = stopwords.words('english')

# 開始進行 out-of-core learning 
#　45次　* 1000 筆資料

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:53


In [9]:
# 用剩下的 5000筆　　測試準確度

X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.866


In [10]:
# 用來測試的那 5000 筆資料當然不能浪費呀
# 繼續拿來更新我們的模型

clf = clf.partial_fit(X_test, y_test)


# 序列化 


In [11]:
#一個程式被載入到記憶體當中運作，那麼在記憶體內的那個資料就被稱為程序(process)。
#在程序運行的過程中，所有的變量都是在記憶體中，比如，定義一個dict：

d = dict(name='Bob', age=20, score=88)

#可以隨時修改變量，比如把name改成'Bill'
#但是一旦程序結束，變量所佔用的記憶體就被操作系統全部回收。
#如果沒有把修改後的'Bill'存儲到硬碟上，下次重新運行程序，變量又被初始化為'Bob'。

#我們把變量從記憶體中變成可存儲或傳輸的過程稱之為序列化，
#在Python中叫pickling，在其他語言中也被稱之為serialization，marshalling，flattening等等

#序列化之後，就可以把序列化後的內容寫入硬碟，或者通過網絡傳輸到別的機器上。

#反過來，把變量內容從序列化的對象重新讀到內存裡稱之為反序列化，即unpickling。

#Python提供兩個模塊來實現序列化：cPickle和pickle。這兩個模塊功能是一樣的，
#區別在於cPickle是C語言寫的，速度快，pickle是純Python寫的，速度慢

In [12]:
import pickle

d = dict(name='Bob', age=20, score=88)
pickle.dumps(d)

#pickle.dumps()方法把任意對象序列化成一個str，然後，就可以把這個str寫入文件。



b'\x80\x03}q\x00(X\x04\x00\x00\x00nameq\x01X\x03\x00\x00\x00Bobq\x02X\x03\x00\x00\x00ageq\x03K\x14X\x05\x00\x00\x00scoreq\x04KXu.'

In [13]:
f = open('dump.txt', 'wb')
pickle.dump(d, f)
f.close()

#寫入的dump.txt文件的內容，這些是Python保存的對象內部信息。

In [14]:
#把內容讀到一個str，然後用pickle.loads()方法反序列化出對象，
#也可以直接用pickle.load()方法從一個file-like Object中直接反序列化出對象。
f = open('dump.txt', 'rb')
d = pickle.load(f)
f.close()
d

{'age': 20, 'name': 'Bob', 'score': 88}

以上是序列化簡介

# 序列化 scikit-learn estimators

In [12]:
import pickle
import os
# 建立一個　movieclassifier　目錄，　pkl_objects　子目錄

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

#  將停用字　以及　分類器　序列化　存在硬碟中 ; protocol=4 : python3.4 中的　pickle協定

pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)   
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

In [14]:
%%writefile movieclassifier/vectorizer.py
# 將向量化的腳本也寫入硬碟
# 要使用的時候　需要載入當下的工作環境之中

from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(
                os.path.join(cur_dir, 
                'pkl_objects', 
                'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

Overwriting movieclassifier/vectorizer.py


In [20]:
# 測試一下能不能正常載入

import pickle
import re
import os
from vectorizer import vect

clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))

In [21]:
import numpy as np
label = {0:'negative', 1:'positive'}
# 測試一下一個簡單的句子
example = ['I love this movie']
# 句子轉成向量　回傳類別標籤以及相對應的機率
X = vect.transform(example)
print('Prediction: %s\nProbability: %.2f%%' %\
      (label[clf.predict(X)[0]], clf.predict_proba(X).max()*100))

Prediction: positive
Probability: 82.18%


# 用　SQLite 　存預測結果的回饋意見

SQLite 　是開放原始碼的資料庫引擎
運作時不需要單獨的伺服器
適合簡單的ｗｅｂ系統
可以理解為　單獨的　資料庫檔案
我們的系統可以直接存取　
在幾乎所有的ＯＳ都可以執行

最重要的是！　　python 的標準函式庫　
包含了 SQLite API : sqlite3


In [23]:
import sqlite3
import os
# 從新建立兩份範例資料　　
if os.path.exists('reviews.sqlite'):
    os.remove('reviews.sqlite')

conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE review_db (review TEXT, sentiment INTEGER, date TEXT)')

example1 = 'I love this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example1, 1))

example2 = 'I disliked this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example2, 0))

conn.commit()
conn.close()

In [24]:
# 檢查是否寫入範例資料
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()

c.execute("SELECT * FROM review_db WHERE date BETWEEN '2017-01-01 10:10:10' AND DATETIME('now')")
results = c.fetchall()

conn.close()

In [25]:
print(results)


[('I love this movie', 1, '2017-07-19 15:42:37'), ('I disliked this movie', 0, '2017-07-19 15:42:37')]
